<a href="https://colab.research.google.com/github/Ro-Cari/FATEC_CIENCIA_DADOS/blob/main/Trabalho_Algebra_Linear(PNL).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Trabalho de Álgebra Linear - Processamento de Linguahem Natural

Indicações de coqueteis utilizando base de dados Kaggle: Conjunto de dados de receitas de bebidas

###Algoritmo de PLN utilizando a Similaridade do Cosseno

In [36]:
import nltk
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from tabulate import tabulate

In [37]:
# Garantir que recursos do NLTK estão disponíveis
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [38]:
# Carrega o dataset de coquetéis
base_coqueteis= pd.read_csv('/content/final_cocktails.csv')

In [39]:
# Exibe as primeiras linhas da base de dados
base_coqueteis.head()

,Unnamed: 0,id,name,alcoholic,category,glassType,instructions,drinkThumbnail,ingredients,ingredientMeasures,text
0,0,0,A1,Alcoholic,Cocktail,Cocktail glass,"Pour all ingredients into a cocktail shaker, m...",https://www.thecocktaildb.com/images/media/dri...,"['Gin', 'Grand Marnier', 'Lemon Juice', 'Grena...","['1 3/4 shot ', '1 Shot ', '1/4 Shot', '1/8 Sh...",question Generate a cocktail with Gin Grand Ma...
1,1,1,ABC,Alcoholic,Shot,Shot glass,Layered in a shot glass.,https://www.thecocktaildb.com/images/media/dri...,"['Amaretto', 'Baileys irish cream', 'Cognac']","['1/3 ', '1/3 ', '1/3 ']",question Generate a cocktail with Amaretto Bai...
2,2,2,Ace,Alcoholic,Cocktail,Martini Glass,Shake all the ingredients in a cocktail shaker...,https://www.thecocktaildb.com/images/media/dri...,"['Gin', 'Grenadine', 'Heavy cream', 'Milk', 'E...","['2 shots ', '1/2 shot ', '1/2 shot ', '1/2 sh...",question Generate a cocktail with Gin Grenadin...
3,3,3,ACID,Alcoholic,Shot,Shot glass,Poor in the 151 first followed by the 101 serv...,https://www.thecocktaildb.com/images/media/dri...,"['151 proof rum', 'Wild Turkey']","['1 oz Bacardi ', '1 oz ']",question Generate a cocktail with 151 proof ru...
4,4,4,Adam,Alcoholic,Ordinary Drink,Cocktail glass,"In a shaker half-filled with ice cubes, combin...",https://www.thecocktaildb.com/images/media/dri...,"['Dark rum', 'Lemon juice', 'Grenadine']","['2 oz ', '1 oz ', '1 tsp ']",question Generate a cocktail with Dark rum Lem...


In [40]:
# Função para tratamento de texto
def tratar_texto(texto):
    texto = texto.lower()
    texto = texto.translate(str.maketrans('', '', string.punctuation))
    palavras = word_tokenize(texto)
    palavras_sem_stopwords = [p for p in palavras if p not in stopwords.words('english')]
    return ' '.join(palavras_sem_stopwords)

In [41]:
# Aplica o tratamento à base
base_coqueteis['texto_tratado'] = base_coqueteis['text'].apply(tratar_texto)

In [42]:
# Função para recomendar coquetéis com base nos ingredientes informados
def recomendar_coqueteis(ingredientes_usuario, base, quantidade=3):
    ingredientes_tratados = tratar_texto(ingredientes_usuario)
    vetorizar = TfidfVectorizer()
    textos = base['texto_tratado'].tolist() + [ingredientes_tratados]
    matriz_tfidf = vetorizar.fit_transform(textos)
    similaridade = cosine_similarity(matriz_tfidf[-1], matriz_tfidf[:-1])
    indices_mais_similares = similaridade[0].argsort()[-quantidade:][::-1]
    return base.iloc[indices_mais_similares][['name', 'ingredients', 'instructions']]

In [46]:
# Solicita os ingredientes do usuário
ingredientes_digitados = input("Digite os ingredientes que você tem (em inglês, separados por espaço): ")

# Obtem recomendações
recomendacoes = recomendar_coqueteis(ingredientes_digitados, base_coqueteis)

# Exibe resultados em tabela
print("\n🍹 Coquetéis recomendados:\n")
print(tabulate(recomendacoes, headers='keys', tablefmt='fancy_grid', showindex=False))

Digite os ingredientes que você tem (em inglês, separados por espaço): rum lime sugar

🍹 Coquetéis recomendados:

╒═══════════════════╤═══════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│ name              │ ingredients                                                                       │ instructions                                                                                                                                                                                │
╞═══════════════════╪═══════════════════════════════════════════════════════════════════════════════════╪═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════